# TASK 1 : Install Dependencies & Run a SparkSession


In [80]:
! pip install pyspark
! pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 943.6 kB/s eta 0:00:00m eta 0:00:010:00:01


In [33]:
#create a sparksession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
sp = SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [4]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.60 KiB | 1.87 MiB/s, done.


In [5]:
#create a spark dataframe
df = sp.read.csv('data/admission_dataset/Admission_Predict_Ver1.1.csv', header=True, inferSchema=True)
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [6]:
#display dataframe
df.head()

Row(Serial No=1, GRE Score=337, TOEFL Score=118, University Rating=4, SOP=4.5, LOR=4.5, CGPA=9.65, Research=1, Chance of Admit=0.92)

In [7]:
#get the no.of rows & columns
print(df.count(), len(df.columns))

500 9


In [8]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [9]:
#get the summary statistics
df.describe().show()

24/02/14 06:45:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [16]:
#drop the unnecessary column 
df = df.drop("Serial No")

In [17]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [68]:
# check for null values
for i in df.columns:
    print(i+":", df[df[i].isNull()].count())

# null_df = df
# null_df = null_df.withColumn("name", lit(None).cast("string"))
# null_df.show()

# null_cols = []
# for col_name in null_df.columns:
#      if null_df.filter(col(col_name).isNull()).count() > 0:
#         null_df = null_df.drop(col_name)
         
df.show()

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0
+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|       

# TASK 4 : Correlation Analysis & Feature Selection

In [72]:
# correlation analysis
for col_name in df.columns:
print('Correlation to chance of admit col for {} is {}'.format(col_name, df.stat.corr(('Chance of Admit', col_name)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [88]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembeler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'CGPA'], outputCol='features')

In [92]:
#display dataframe
output_data = assembeler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [97]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features', 'Chance of Admit')

In [99]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [101]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [110]:
#build & train the model
models = LinearRegression(featuresCol='features', labelCol='Chance of Admit')
model = models.fit(train)

24/02/14 09:38:35 WARN Instrumentation: [a838d677] regParam is zero, which might cause numerical instability and overfitting.


In [108]:
#get coefficients & intercept
print("coefficients", model.coefficients)
print("intercept", model.intercept)

coefficients [0.002158647016101521,0.003420766092937167,0.14129332845744827]
intercept -1.5373328405162217


In [113]:
#get summary of the model
summary = model.summary

In [116]:
#print the rmse & r2 score
print('RMSE', summary.rootMeanSquaredError)
print('r2 score', summary.r2)

RMSE 0.06031567966227178
r2 score 0.8134489010049327


# TASK 6 : Evaluate & Save the Model

In [117]:
#transform on the test data
predictions = model.transform(test)

In [121]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,100.0,7.56]|           0.47|  0.498928966585245|
|  [295.0,93.0,7.2]|           0.46|0.43491124077051113|
| [295.0,99.0,7.65]|           0.57| 0.5190178351339858|
| [297.0,96.0,7.43]|           0.34|0.48198829862673875|
|[297.0,101.0,7.67]|           0.57| 0.5330025279212123|
| [298.0,92.0,7.88]|           0.51| 0.5340458790769431|
| [299.0,96.0,7.86]|           0.54| 0.5470617238956446|
| [299.0,97.0,7.66]|           0.38| 0.5222238242970922|
|[299.0,100.0,7.88]|           0.51| 0.5635706548365422|
|[299.0,100.0,7.89]|           0.59| 0.5649835881211165|
|[299.0,102.0,8.62]|           0.56| 0.6749692500809279|
| [300.0,95.0,8.22]|           0.62| 0.5966652030634902|
| [300.0,99.0,8.01]|           0.58| 0.5806766684591746|
|[300.0,100.0,8.26]|           0.62| 0.6194207666664742|
| [301.0,96.0,7.56]|           

In [127]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator 
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit', metricName='r2')
print('r2 on the test data', evaluator.evaluate(predictions))

r2 on the test data 0.7816299572049134


In [128]:
#save the model
model.save("uni-predicition-model")

In [130]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('uni-predicition-model')